In [83]:
from matplotlib import use
use('Qt4Agg')

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from os import listdir
import cv2

In [84]:
# useful constants 
pi = np.pi
halfPi = pi/2.0
halfRangeDivPi = 255.0/pi ###WRONG
sqrtTwo = np.sqrt(2.0)
infFloat = float('inf')
sat = 0.7

In [85]:
#import image data from .emd files as type Signal2D using hyperSpy
datadir = '/Users/matt/Box/KEM Group Docs/For Matt/Datasets/180911/Processed Data/DPC/'
name = '12_DPC_1024px_1x2.5us_CL380mm 29.1 kx 0001'
emdf = hs.load(datadir+name+'.emd')

# emdf

In [20]:
emdf[5].data

array([[44123., 42690., 47518., ..., 33667., 34512., 37571.],
       [43558., 43542., 45371., ..., 34173., 34053., 37356.],
       [46833., 47039., 44220., ..., 36632., 35578., 36616.],
       ...,
       [18958., 18526., 18111., ..., 10468., 10705., 10565.],
       [17980., 17873., 18953., ..., 10826., 10505., 10931.],
       [19170., 17063., 17950., ..., 11720., 10651., 11557.]])

In [4]:
# Analyze .emd file tree strucutre using hyperSpy
# emdf
emdf[0].original_metadata

├── Acquisition
│   ├── AcquisitionDatetime
│   │   └── DateTime = 0
│   ├── AcquisitionStartDatetime
│   │   └── DateTime = 1536704557
│   ├── BeamType = 
│   └── SourceType = Monochromator
├── BinaryResult
│   ├── AcquisitionUnit = 
│   ├── CompositionType = 
│   ├── Detector = DF4
│   ├── DetectorIndex = 3
│   ├── Encoding = 
│   ├── Offset
│   │   ├── x = -4.3129248712936381e-007
│   │   └── y = -4.3129248712936381e-007
│   ├── PixelSize
│   │   ├── height = 8.4236813892453869e-010
│   │   └── width = 8.4236813892453869e-010
│   ├── PixelUnitX = m
│   └── PixelUnitY = m
├── Core
│   ├── MetadataDefinitionVersion = 7.9
│   ├── MetadataSchemaVersion = v1/2013/07
│   └── guid = 00000000000000000000000000000000
├── CustomProperties
│   ├── Aperture[C1].Name
│   │   ├── type = string
│   │   └── value = 2000
│   ├── Aperture[C2].Name
│   │   ├── type = string
│   │   └── value = 50
│   ├── Aperture[C3].Name
│   │   ├── type = string
│   │   └── value = 2000
│   ├── Aperture[OBJ].Name
│ 

In [86]:
# create differential readouts in ndArray format
AC = np.empty((1024,1024), dtype=np.float32)
AC = np.subtract(emdf[4].data,emdf[0].data)      #.data attribute allows for export to ndArray

BD = np.empty((1024,1024), dtype=np.float32)
BD = np.subtract(emdf[5].data,emdf[3].data)

In [ ]:
# calculate phase using vectorized trick for quotient
# quoTrick = np.empty((1024,1024), dtype=np.float32)
# IA = np.zeros_like(AC)

# with np.nditer(IA, op_flags=['readwrite']) as it:
#     for x in it:
#         x[...] = float('inf')
        
# quoTrick = np.divide(AC, BD, out= IA, where= BD !=0)
# phaseTrick = np.arctan(quoTrick)           #how did you get this to work last night???

In [87]:
# calculate phase using laborius checking technique
quoHard = np.empty((1024,1024), dtype=np.float32)
phaseHard = np.empty((1024,1024), dtype=np.float32)

for i in range(0,1024):                                
    for j in range (0,1024):                           
            if BD[i][j] == 0:                           #handle divide by zero cases
                quoHard[i][j] = float('inf')
            else:
                quoHard[i][j] = (AC[i][j])/(BD[i][j])   # (y/x) where AC = y, BD = x 
                
            if BD[i][j]<1:
                phaseHard[i][j] = np.arctan(quoHard[i][j])
                phaseHard[i][j] += pi
                if BD[i][j] == 0 and AC[i][j] > 0:
                    phaseHard[i][j] += pi
            else:
                phaseHard[i][j] = np.arctan(quoHard[i][j])
            
            phaseHard[i][j] = (phaseHard[i][j] + pi/2.0)*(179/pi)

In [ ]:
# show a difference map of the arctan of each method


In [88]:
magnitude = np.sqrt(np.add(np.square(AC),np.square(BD)))

In [ ]:
# explore the properties of magnitude to properly normalize the dataset 
# np.max(magnitude)
# np.min(magnitude)

# for i in range(0,1024):                                
#     for j in range (0,1024): 
#         if i%100 == 0:
#             print(magnitude[i][j])

In [89]:
# normalize magnitude to fit into hsv 
#For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]
norm = np.max(magnitude)
for i in range(0,1024):                                
    for j in range (0,1024): 
        magnitude[i][j] = (magnitude[i][j] * (0.7/norm)) + 0.3

In [90]:
dpcmap = np.empty((1024,1024,3),dtype=np.float32)

for i in range(0,1024):                                
    for j in range (0,1024): 
        dpcmap[i][j][0] = phaseHard[i][j]
        dpcmap[i][j][1] = 1.0
        dpcmap[i][j][2] = magnitude[i][j]

In [91]:
dpcOut = np.empty((1024,1024,3),dtype=np.float32)
dpcOut = cv2.cvtColor(dpcmap, cv2.COLOR_HSV2BGR )

In [92]:
plt.imshow(dpcOut)
plt.axis('off')
plt.margins(0,0)
# plt.savefig(name+'_cmap.png')
plt.savefig(name+'cmap.eps', format='eps', dpi=1000,
bbox_inches = 'tight')
plt.show()

# gca().set_axis_off()
# subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
#             hspace = 0, wspace = 0)
# margins(0,0)
# gca().xaxis.set_major_locator(NullLocator())
# gca().yaxis.set_major_locator(NullLocator())
# savefig("filename.pdf", bbox_inches = 'tight',
#     pad_inches = 0)
